![image](./asset/overview/overview.png)

In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [7]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [18]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [19]:
# Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print("length of splits: ", len(splits))

length of splits:  66


In [20]:
# Embed Documents
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(
        model="text-embedding-3-small",
        openai_api_key=OPENAI_API_KEY,
    )
)

In [42]:
retriever = vectorstore.as_retriever()

In [44]:
# Retrieval and generation
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


c:\Users\user\anaconda3\envs\rag-from-scratch\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [46]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000200D27740D0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000200D238DA50> root_client=<openai.OpenAI object at 0x00000200D2208E10> root_async_client=<openai.AsyncOpenAI object at 0x00000200D2775F10> temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''


In [47]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [48]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [49]:
# Question
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique that breaks down complex problems into smaller, more manageable steps. It can be achieved through various methods such as using LLM with simple prompting, task-specific instructions, or human inputs. The process involves generating multiple thoughts per step to create a tree structure for problem-solving.'

### Indexing

![image](./asset/overview/indexing.png)

In [1]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [3]:
import tiktoken

def num_tokens_from_string(string:str, encoding_name: str, debug: bool = False) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    if debug:
        print("encoding string: ", encoding.encode(string))
    
    return num_tokens

num_tokens_from_string(question, "cl100k_base", True)

encoding string:  [3923, 13124, 315, 26159, 656, 358, 1093, 30]


8

In [9]:
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)

In [13]:
print("length of query_result: ", len(query_result))
print("length of document_result: ", len(document_result))

length of query_result:  1536
length of document_result:  1536


Cosine similarity is reccomended (1 indicates identical) for OpenAI embeddings.

In [14]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity: ", similarity)

Cosine Similarity:  0.8807044730847652


### Retrieval

In [23]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("What is Task Decomposition?")
print(len(docs))

c:\Users\user\anaconda3\envs\rag-from-scratch\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


1


### Generation

![image](./asset/overview/generation.png)

In [24]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))]


In [25]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chain
chain = prompt | llm

In [26]:
# Run
chain.invoke({"context": docs, "question": "What is Task Decomposition?"})

AIMessage(content='Task decomposition is the process of breaking down a problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure. It can be done using methods such as LLM with simple prompting, task-specific instructions, or human inputs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 203, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c24e1874-6f0f-491c-9e7d-70aedc028db9-0', usage_metadata={'input_tokens': 203, 'output_tokens': 48, 'total_tokens': 251})